## 자료구조

### 큐 
+ 인큐(Enqueue)
    + `rear`에 인큐 -> 복잡도: O(1)
+ 디큐(Dequeue)
    + `front`에 디큐 -> 복잡도: O(n)
    + 모든 원소를 앞쪽으로 옮겨야 됨
    + 비효율적임

### 링 버퍼로 큐 구현하기  
+ `front`와 `rear`를 업데이트하는 것만으로 인큐와 디큐 수행 가능
+ 모든 처리의 복잡도: O(1)

In [5]:
# 고정 길이 큐 클래스 FixedQueue구현하기
from typing import Any

class FixedQueue:
    '''고정길이 큐 클래스'''
    
    class Empty(Exception):
        '''비어 있는 FixedQueue에서 디큐 또는 피크할 때 내보내는 예외처리'''
        pass
    
    class Full(Exception):
        '''꽉 찬 FixedQueue에 인큐할 때 내보내는 예외 처리'''
        pass
    
    def __init__(self,capacity:int)->None:
        '''큐 초기화'''
        self.now=0 # 현재 데이터 개수
        self.front=0 # 맨 앞 원소 포인터
        self.rear=0 # 맨 끝 원소 포인터
        self.capacity=capacity # 큐의 크기
        self.que=[None]*capacity # 본체
        
    def __len__(self)->int:
        '''큐에 있는 모든 데이터 개수 반환'''
        return self.now
    
    def is_empty(self)->bool:
        '''큐가 비어있는지 확인'''
        return self.now<=0
        # 큐가 비어있을 때 front==rear
    
    def is_full(self)->bool:
        '''큐가 꽉 차있는지 확인'''
        return self.now>=self.capacity
        # 큐가 꽉 차있을 때 front==rear

    def enqueue(self,value:Any)->None:
        '''큐의 rear에 value  인큐'''
        if self.is_full(): # 큐가 가득 차있는 경우
            raise FixedQueue.Full # 예외 처리 발생
        self.que[self.rear]=value
        self.rear+=1
        self.now+=1
        if self.rear==self.capacity: # rear가 배열 인덱스 범위를 초과할 때
            self.rear=0
    
    def dequeue(self)->Any:
        '''큐의 front에서 디큐'''
        if self.is_empty(): # 큐가 비어있는 경우
            raise FixedQueue.Empty # 예외 처리
        value=self.que[self.front]
        self.front+=1
        self.now-=1
        if self.front==self.capacity: # front가 배열 인덱스 범위를 초과할 때
            self.front=0
        return value
    
    def peek(self)->Any:
        '''큐의 front에서 데이터 피크'''
        if self.is_empty():
            raise FixedQueue.Empty
        return self.que[self.front]
    
    def clear(self)->None:
        '''큐의 모든 데이터 비움'''
        self.now=self.front=self.rear=0
    
    def find(self,value:Any)->int:
        '''큐에서 value찾아 인덱스 반환'''
        for i in range(self.now):
            idx=(i+self.front)%self.capacity
            if self.que[idx]==value:
                return idx #검색 성공
        return -1 #검색 실패
    
    def count(self,value:Any)->int:
        '''큐에 있는 value개수를 반환'''
        cnt=0
        for i in range(self.now):
            idx=(i+self.front)%self.capacity
            if self.que[idx]==value:
                cnt+=1
        return cnt
    
    def __contains__(self,value:Any)->bool:
        '''큐에 value가 있는지 판단'''
        return self.count(value)>0
    
        
    def dump(self)->None:
        '''큐의 모든 데이터를 front->rear 순으로 출력'''
        if self.is_empty():
            print('큐가 비어있습니다.')
        else:
            for i in range(self.now):
                idx=(i+self.front)%self.capacity
                print(self.que[idx],end=' ')
            print()       

+ `enqueue()` 함수  
    + `rear`에 value 추가
    + `rear` += 1
    + `rear`가 배열 인덱스 범위 초과했을 경우(`rear`==`capacity`)  
        `rear`==0
+ `dequeue()` 함수
    + `front` +=1
    + `front`가 배열 인덱스 범위 초과했을 경우(`front`==`capacity`)  
        `front`==0

+ `find()` 함수
    + `front` -> `rear`로 순회
    + iter: 0 -> 1 -> 2 -> 3 -> 4
    + idx : 3 -> 4 -> 0 -> 1 -> 2 
    + idx = (i+`front`)%`capacity`
    

### 큐 프로그램 만들기

In [3]:
from enum import Enum

Menu=Enum('Menu',['ENQUEUE','DEQUEUE','PEEK','SEARCH','DUMP','OFF'])

def select_menu()->Menu:
    '''메뉴 선택'''
    s=[f'({m.value}){m.name}' for m in Menu]
    while True:
        print(*s)
        n=int(input('메뉴 선택: '))
        if 1<=n<=len(Menu):
            return Menu(n)

In [6]:
q=FixedQueue(64)

while True:
    print('-'*30)
    print(f'현재 데이터 개수 : {q.now}/{q.capacity}')
    menu=select_menu()
    
    if menu==Menu.ENQUEUE:
        x=int(input('데이터 입력: '))
        try:
            q.enqueue(x)
        except FixedQueue.Full:
            print('큐가 가득 찼습니다.')
    
    elif menu==Menu.DEQUEUE:
        try:
            x=q.dequeue()
            print(f'디큐한 데이터는 {x}입니다.')
        except FixedQueue.Empty:
            print('큐가 비어있습니다.')
    
    elif menu==Menu.PEEK:
        try:
            x=q.peek()
            print(f'피크한 데이터는 {x}입니다.')
        except FixedQueue.Empty:
            print('큐가 비어있습니다.')
            
    elif menu==Menu.SEARCH:
        x=int(input('검색할 데이터 입력: '))
        if x in q:
            print(f'{q.count(x)}개 존재하고, 맨 앞의 위치는 {q.find(x)}입니다.')
        else:
            print('찾을 수 없습니다.')
    
    elif menu==Menu.DUMP:
        q.dump()
    
    else:
        print('프로그램 종료') 
        break           

------------------------------
현재 데이터 개수 : 0/64
(1)ENQUEUE (2)DEQUEUE (3)PEEK (4)SEARCH (5)DUMP (6)OFF
------------------------------
현재 데이터 개수 : 1/64
(1)ENQUEUE (2)DEQUEUE (3)PEEK (4)SEARCH (5)DUMP (6)OFF
------------------------------
현재 데이터 개수 : 2/64
(1)ENQUEUE (2)DEQUEUE (3)PEEK (4)SEARCH (5)DUMP (6)OFF
------------------------------
현재 데이터 개수 : 3/64
(1)ENQUEUE (2)DEQUEUE (3)PEEK (4)SEARCH (5)DUMP (6)OFF
------------------------------
현재 데이터 개수 : 4/64
(1)ENQUEUE (2)DEQUEUE (3)PEEK (4)SEARCH (5)DUMP (6)OFF
------------------------------
현재 데이터 개수 : 5/64
(1)ENQUEUE (2)DEQUEUE (3)PEEK (4)SEARCH (5)DUMP (6)OFF
1 3 5 6 7 
------------------------------
현재 데이터 개수 : 5/64
(1)ENQUEUE (2)DEQUEUE (3)PEEK (4)SEARCH (5)DUMP (6)OFF
디큐한 데이터는 1입니다.
------------------------------
현재 데이터 개수 : 4/64
(1)ENQUEUE (2)DEQUEUE (3)PEEK (4)SEARCH (5)DUMP (6)OFF
디큐한 데이터는 3입니다.
------------------------------
현재 데이터 개수 : 3/64
(1)ENQUEUE (2)DEQUEUE (3)PEEK (4)SEARCH (5)DUMP (6)OFF
5 6 7 
-------------------------